In [ ]:
import numpy as np
import elboflow as ef
import tensorflow as tf
import scipy.stats
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt, patches as mpatches
%matplotlib inline

In [ ]:
# Generate some data for mixtures
np.random.seed(6)
num_samples = 100
num_dims = 2
num_components = 5

component_means = np.random.normal(0, 1, (num_components, num_dims))
component_precisions = np.random.gamma(100, 1, (num_components, num_dims))
z = np.random.randint(num_components, size=num_samples)
x = component_means[z] + np.random.normal(0, 1, (num_samples, num_dims)) / np.sqrt(component_precisions[z])

fig, ax = plt.subplots(1, 1)
pts = ax.scatter(*x.T, c=z, cmap='Vega10', marker='.')
ax.set_aspect('equal')
plt.colorbar(pts)

In [ ]:
class GaussianMixtureModel(ef.Model):
    def __init__(self, x, num_components, create_optimizer=None, create_session=None):
        self.num_components = num_components
        super(GaussianMixtureModel, self).__init__(None, [x], create_optimizer, create_session)
        
    def setup(self, x):
        n, p = x.shape
        
        # Define the factors
        q_mu = ef.MultiNormalDistribution(
            ef.get_variable('mu_mean', (1, self.num_components, p)),
            ef.get_positive_definite_variable('mu_precision', (1, self.num_components, p, p))
        )
        q_tau = ef.WishartDistribution(
            ef.get_positive_variable('tau_dof', (1, self.num_components)) + float(p - 1),
            ef.get_positive_definite_variable('tau_scale', (1, self.num_components, p, p)),
        )
        q_z = ef.CategoricalDistribution(
            ef.get_normalized_variable('z_p', (n, self.num_components))
        )
        
        # Evaluate the likelihood of all points given all possible component assignments
        log_gaussian_likelihood = ef.MultiNormalDistribution.log_likelihood(x[:, None, :], q_mu, q_tau)
        # Compute the aggregate likelihood
        log_mixture_likelihood = ef.CategoricalDistribution.mixture_log_likelihood(q_z, log_gaussian_likelihood)

        return tf.reduce_sum(log_mixture_likelihood), {
            'mu': q_mu,
            'tau': q_tau,
            'z': q_z,
        }, {
            'mu': ef.NormalDistribution(0.0, 1e-3).log_proba,
            'tau': ef.WishartDistribution(2.0, .2 * np.eye(num_dims)).log_proba,
            'z': ef.CategoricalDistribution(np.ones((1, num_components), np.float32) / num_components).log_proba
        }
    
model = GaussianMixtureModel(x, num_components)
model.run(model.elbo)

In [ ]:
trace = model.optimize(10000, model.loss, tqdm=tqdm_notebook)
plt.plot(trace[model.loss])
plt.yscale('log')

In [ ]:
fig, ax = plt.subplots(1, 1)
pts = ax.scatter(*x.T, c=z, cmap='Vega10', marker='.')

_means = model.run(model['mu'].mean)
_covs = np.linalg.inv(model.run(model['tau'].mean))
for i in range(num_components):
    xy = _means[0, i]
    _cov = _covs[0, i]
    ax.add_artist(ef.ellipse_from_cov(xy, _cov, facecolor='none', edgecolor='r'))
    
ax.set_aspect('equal')
plt.colorbar(pts)